In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

load nci


In [4]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device,
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/1005 [00:00<?, ?it/s]

epoch:   0 loss:0.762612 auc:0.5275
epoch:  20 loss:0.574920 auc:0.7285
epoch:  40 loss:0.458052 auc:0.7237
epoch:  60 loss:0.377173 auc:0.7225
epoch:  80 loss:0.345345 auc:0.7225
epoch: 100 loss:0.331268 auc:0.7285
epoch: 120 loss:0.323708 auc:0.7237
epoch: 140 loss:0.319147 auc:0.7297
epoch: 160 loss:0.316135 auc:0.7309
epoch: 180 loss:0.314027 auc:0.7321
epoch: 200 loss:0.312497 auc:0.7321
epoch: 220 loss:0.311360 auc:0.7321
epoch: 240 loss:0.310500 auc:0.7321
epoch: 260 loss:0.309848 auc:0.7309
epoch: 280 loss:0.309348 auc:0.7273


  0%|          | 1/1005 [00:04<1:13:42,  4.40s/it]

epoch: 300 loss:0.308960 auc:0.7273
Fit finished.
epoch:   0 loss:0.769195 auc:0.4000
epoch:  20 loss:0.568127 auc:0.6560
epoch:  40 loss:0.449948 auc:0.6743
epoch:  60 loss:0.378639 auc:0.6709
epoch:  80 loss:0.347310 auc:0.6491
epoch: 100 loss:0.332720 auc:0.6366
epoch: 120 loss:0.324514 auc:0.6297
epoch: 140 loss:0.319364 auc:0.6229


  0%|          | 2/1005 [00:07<56:43,  3.39s/it]  

...

epoch: 180 loss:0.318300 auc:0.6049
Fit finished.
epoch:   0 loss:0.823017 auc:0.6354
epoch:  20 loss:0.592827 auc:0.7240
epoch:  40 loss:0.504344 auc:0.6997
epoch:  60 loss:0.413213 auc:0.7691
epoch:  80 loss:0.370259 auc:0.7483
epoch: 100 loss:0.348647 auc:0.7378
epoch: 120 loss:0.336349 auc:0.7413
epoch: 140 loss:0.328594 auc:0.7361
epoch: 160 loss:0.323346 auc:0.7465
epoch: 180 loss:0.319642 auc:0.7448
epoch: 200 loss:0.316952 auc:0.7448


100%|█████████▉| 1004/1005 [6:56:32<00:17, 17.33s/it]

epoch: 220 loss:0.314951 auc:0.7483
Fit finished.
epoch:   0 loss:0.805885 auc:0.5312
epoch:  20 loss:0.578639 auc:0.5472
epoch:  40 loss:0.484034 auc:0.4912
epoch:  60 loss:0.405671 auc:0.4304
epoch:  80 loss:0.364047 auc:0.4272
epoch: 100 loss:0.342703 auc:0.4128
epoch: 120 loss:0.331317 auc:0.4144
epoch: 140 loss:0.324650 auc:0.4144


100%|██████████| 1005/1005 [6:56:37<00:00, 24.87s/it]

epoch: 160 loss:0.320395 auc:0.4144
Fit finished.


In [7]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")